# Silver Layer: Data Cleaning & Transformation

## Purpose
Apply data quality rules and standardization to Bronze layer raw data, producing clean, validated datasets ready for analytics.

## What This Notebook Does
- **Reads** Bronze layer tables (raw ingested data)
- **Cleans** data through trimming whitespace and normalizing values
- **Validates** data quality by removing records with missing critical fields (Customer ID)
- **Standardizes** column names for consistency across the lakehouse
- **Performs** sanity checks on both source and target datasets
- **Writes** validated data to Silver layer in Delta format

## Transformations Applied
- String trimming to remove leading/trailing spaces
- Data normalization for consistent formatting
- Null/missing value handling for mandatory fields
- Column renaming to follow naming conventions

## Quality Checks
- Pre-transformation: DataFrame structure and row count validation
- Post-transformation: Silver table integrity verification

## Output
Clean, validated Delta tables in `workspace.silver` schema

#Initialization

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

# Read Bronze table

In [0]:
df = spark.table("workspace.bronze.crm_cust_info")

#Silver Transformations

##Trimming

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

##Normalization

In [0]:

df = (
    df
    .withColumn(
        "cst_marital_status",
        F.when(F.upper(F.col("cst_marital_status")) == "S", "Single")
         .when(F.upper(F.col("cst_marital_status")) == "M", "Married")
         .otherwise("n/a")
    )
    .withColumn(
        "cst_gndr",
        F.when(F.upper(F.col("cst_gndr")) == "F", "Female")
         .when(F.upper(F.col("cst_gndr")) == "M", "Male")
         .otherwise("n/a")
    )
)

##Remove Records with Missing Customer ID

In [0]:
df = df.filter(col("cst_id").isNotNull())

## Renaming Columns

In [0]:
RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_number",
    "cst_firstname": "first_name",
    "cst_lastname": "last_name",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "created_date"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

## Sanity checks of dataframe

In [0]:
df.limit(10).display()

#Writing Silver Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_customers")

## Sanity checks of silver table

In [0]:
%sql
SELECT * FROM workspace.silver.crm_customers LIMIT 10